**`__init__` method**: The constructor initializes the neural network model by calling the `initialize_parameters` method to initialize the weights and biases.

**`initialize_parameters` method**: This method initializes the parameters of the neural network (weights and biases) with random values. It takes a list `layer_dims` as input, which specifies the number of nodes in each layer of the network.

**`linear_forward` method**: This method performs the linear transformation step of a single layer in the neural network. It takes the input activations `A_prev`, weight matrix `W`, and bias vector `b` as inputs, and computes the linear combination `Z = W.T * A_prev + b`.

**`L_layer_forward` method**: This method implements the forward propagation step of the entire neural network. It iterates through each layer of the network, applies the linear transformation followed by the activation function (in this case, the sigmoid function), and computes the output activations `A`. It returns the final output activations `A` and the activations of the previous layer `A_prev`.

**`update_parameters` method**: This method updates the parameters (weights and biases) of the neural network using the gradient descent algorithm. It takes the true target value `y`, predicted value `y_hat`, activations of the first layer `A1`, input data `X`, and learning rate `learning_rate` as inputs. It computes the gradients of the loss function with respect to the parameters and updates the parameters accordingly.

**`train` method**: This method trains the neural network using the provided training data (`X_train`, `y_train`). It iterates through multiple epochs, where each epoch consists of iterating through each training example, computing the predicted output, updating the parameters, and calculating the loss. It prints the loss after each epoch.

In [1]:
import numpy as np

class NeuralNetworkRegression:
    def __init__(self, layer_dimensions):
        self.parameters = self.initialize_parameters(layer_dimensions)
    
    def initialize_parameters(self, layer_dimensions):
        np.random.seed(3)
        parameters = {}
        num_layers = len(layer_dimensions)

        for l in range(1, num_layers):
            input_dim = layer_dimensions[l-1]
            output_dim = layer_dimensions[l]
            parameters['W' + str(l)] = np.ones((input_dim, output_dim)) * 0.1
            parameters['b' + str(l)] = np.zeros((output_dim, 1))

        return parameters

    def linear_forward(self, previous_activations, weights, biases):
        linear_output = np.dot(weights.T, previous_activations) + biases
        return linear_output

    def L_layer_forward(self, input_data):
        activations = input_data
        num_layers = len(self.parameters) // 2 

        for l in range(1, num_layers + 1):
            previous_activations = activations
            weights = self.parameters['W' + str(l)]
            biases = self.parameters['b' + str(l)]
            activations = self.linear_forward(previous_activations, weights, biases)
            
        return activations, previous_activations

    def update_parameters(self, target_value, predicted_value, previous_activations, input_data, learning_rate=0.001):
        # Update parameters for the output layer
        self.parameters['W2'][0][0] += learning_rate * 2 * (target_value - predicted_value) * previous_activations[0][0]
        self.parameters['W2'][1][0] += learning_rate * 2 * (target_value - predicted_value) * previous_activations[1][0]
        self.parameters['b2'][0][0] += learning_rate * 2 * (target_value - predicted_value)

        # Update parameters for the hidden layer
        self.parameters['W1'][0][0] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][0][0] * input_data[0][0]
        self.parameters['W1'][0][1] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][0][0] * input_data[1][0]
        self.parameters['b1'][0][0] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][0][0]

        self.parameters['W1'][1][0] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][1][0] * input_data[0][0]
        self.parameters['W1'][1][1] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][1][0] * input_data[1][0]
        self.parameters['b1'][1][0] += learning_rate * 2 * (target_value - predicted_value) * self.parameters['W2'][1][0]

    def train(self, training_data, target_values, epochs=5, learning_rate=0.001):
        for i in range(epochs):
            losses = []

            for j in range(training_data.shape[0]):
                input_data = training_data[j].reshape(2, 1)
                target_value = target_values[j][0]

                predicted_value, previous_activations = self.L_layer_forward(input_data)
                predicted_value = predicted_value[0][0]

                self.update_parameters(target_value, predicted_value, previous_activations, input_data, learning_rate)

                losses.append((target_value - predicted_value) ** 2)

            print('Epoch - ', i + 1, 'Loss - ', np.array(losses).mean())

        return self.parameters

In [2]:
# Instantiate the NeuralNetworkRegression class
layer_dims = [2, 2, 1]
nn_regression = NeuralNetworkRegression(layer_dims)

# Define the input features (X_train) and target variable (y_train)
X_train = np.array([[8, 8], [7, 9], [6, 10], [5, 12]])
y_train = np.array([[4], [5], [6], [7]])

# Train the model
trained_parameters = nn_regression.train(X_train, y_train)

# Print the trained parameters
print("Trained Parameters:")
for key, value in trained_parameters.items():
    print(key, ":", value)

Epoch -  1 Loss -  26.28249792398698
Epoch -  2 Loss -  19.438253848220803
Epoch -  3 Loss -  10.13987443582752
Epoch -  4 Loss -  3.385561305106485
Epoch -  5 Loss -  1.3198454128484565
Trained Parameters:
W1 : [[0.273603   0.3993222 ]
 [0.28787155 0.42586102]]
b1 : [[0.02885522]
 [0.03133223]]
W2 : [[0.42574893]
 [0.50219328]]
b2 : [[0.11841278]]
